In [1]:
import polars as pl

In [25]:
# calculate the average score on the training set
avg_score = (
    pl.scan_csv("/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv")
    .select(pl.col("score").mean())
    .collect()
    .item()
)
# create a submission with the average score for every essay
submission = (
    pl.scan_csv("/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv")
    .select(pl.col("id").unique(), pl.lit(avg_score).alias("score"))
    .collect()
)

In [27]:
# write the submission file
submission.write_csv("submission.csv")